# Business Analysis
This dataset contains data about an undisclosed Indian company and its employees. Features of the dataset include the highest education level of its employees, year of joining, city where the employee is based out of, age and gender of the employees, a binary column of if an employee has ever been 'benched' (temporarily assigned no work), and whether the employee left the company or not. This last feature is the variable we are trying to predict (dependent variable), using all of the previous data as the predictor variables. This company was trying to determine what factors are indicative of if an employee chooses to leave the company or not. This can of course be useful to other companies who are trying to increase their employee retention rate. In order for an predictive algorithm to be considered useful, there should be a statistically significant correlation between the dependent and independent variables. Alternatively, we can use this data to make decision trees that could help classify whether 


Dataset: https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset